In [31]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re, string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
fname = 'tweets.txt'

In [32]:
lines = [line.rstrip() for line in open(fname)]
file = open("newfile.txt", "w")

In [33]:
def checkTopic (topic, text):
    if (set(topic) & set(text)):
        return True
    return False

def bag_of_words(words):
    if len(words) > 0:
        return dict([(word, True) for word in words])

stop = stopwords.words('english')

In [34]:
json_input = lines[4]
decoded = json.loads(json_input)
    
# pretty printing of json-formatted string
print json.dumps(decoded, sort_keys=True, indent=4)

nf_timestamp = decoded['timestamp_ms']
print "nf_timestamp", nf_timestamp

nf_user = decoded['user']['screen_name']
print "nf_user", nf_user

{
    "contributors": null, 
    "coordinates": null, 
    "created_at": "Sun Jun 12 18:07:18 +0000 2016", 
    "entities": {
        "hashtags": [], 
        "symbols": [], 
        "urls": [], 
        "user_mentions": []
    }, 
    "favorite_count": 0, 
    "favorited": false, 
    "filter_level": "low", 
    "geo": null, 
    "id": 742055693281288200, 
    "id_str": "742055693281288193", 
    "in_reply_to_screen_name": null, 
    "in_reply_to_status_id": null, 
    "in_reply_to_status_id_str": null, 
    "in_reply_to_user_id": null, 
    "in_reply_to_user_id_str": null, 
    "is_quote_status": false, 
    "lang": "en", 
    "place": null, 
    "retweet_count": 0, 
    "retweeted": false, 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "text": "Thread of my fav mutuals \ud83d\udc96", 
    "timestamp_ms": "1465754838663", 
    "truncated": false, 
    "user": {
        "contributors_enabled": false, 
        "created_at":

In [35]:
total = 0
exception = 0
topic = ["terrorism"]
print topic
corpus = []
label = []
count = 0
counts = []
words = []

for index in range(len(lines)):
    json_input = lines[index]
    decoded = json.loads(json_input)
    
    try:

        nf_lang = decoded['user']['lang']
        #print "nf_lang", nf_lang
    
        if nf_lang == 'en':
            #print "JSON parsing example: ", decoded['hashtags']
            # cal treure entities and stop words
            nf_tweet_text = decoded['text']
            #print "nf_tweet_text", nf_tweet_text
            nf_RT = (decoded['retweet_count']) > 0
            #print "nf_RT", nf_RT
            if nf_RT:
                nf_rt_text = decoded['retweeted_status']['text']
                nf_tweet_text = nf_rt_text
            #print "nf_tweet_text :: ", nf_tweet_text

            # not alphabetical characters
            nf_tweet_text = re.sub(r"[^a-zA-Z]", " ", nf_tweet_text)
            
            s = nf_tweet_text.split()          
            #print(s)
 
            # filtered = [word for words in s if word not in stopwords.words('english')]
            nf_tweet_filtered_text = [i for i in s if i not in stop]
            #print "nf_tweet_filtered_text :: ", nf_tweet_filtered_text
            nf_tweet_filtered_text = [element.lower() for element in nf_tweet_filtered_text]
            
            # check the topic
            aux = " ".join( nf_tweet_filtered_text )
            corpus.append(aux)
            count = count + 1
            nf_positiu = checkTopic(topic, nf_tweet_filtered_text)
            if nf_positiu:
                total = total + 1
                label.append(1)
                print index, count, total, "**", aux
            else:
                label.append(0)
            counts.append(len(nf_tweet_text)) 
            words.append(len(nf_tweet_filtered_text))

               
            nf_words = []
        
            nf_count_hashtags = len(decoded['entities']['hashtags'])
            #print "nf_count_hastags", nf_count_hashtags
            nf_pos_first_hashtag = 0
            nf_tweet_hashtags = []
            if nf_count_hashtags > 0:
                nf_pos_first_hashtag = decoded['entities']['hashtags'][0]['indices'][0]
                #print "nf_pos_first_hashtag", nf_pos_first_hashtag
                for i in range(nf_count_hashtags):
                    nf_tweet_hashtags.append(decoded['entities']['hashtags'][i]['text'])
                #print "nf_tweet_hashtags", nf_tweet_hashtags
                nf_tweet_hashtags = [element.lower() for element in nf_tweet_hashtags]
                nf_words.append(nf_tweet_hashtags)
        
            nf_count_urls = len(decoded['entities']['urls'])
            #print "nf_count_urls", nf_count_urls
            nf_pos_first_url = 0
            nf_tweet_urls = []
            if nf_count_urls > 0:
                nf_pos_first_url = decoded['entities']['urls'][0]['indices'][0]
                #print "nf_pos_first_url", nf_pos_first_url
                for i in range (nf_count_urls):
                    nf_tweet_urls.append(decoded['entities']['urls'][i]['url'])
                #print nf_tweet_urls
                #nf_tweet_urls = [element.lower() for element in nf_tweet_urls]
                #nf_words.append(nf_tweet_urls)
        
            nf_count_mentions = len(decoded['entities']['user_mentions'])
            #print "nf_count_mentions", nf_count_mentions
            nf_pos_first_mention = 0
            nf_tweet_mentions = []
            if nf_count_mentions > 0:
                nf_pos_first_mention = decoded['entities']['user_mentions'][0]['indices'][0]
                #print "nf_pos_first_mention", nf_pos_first_mention    
                for i in range(nf_count_mentions):
                    nf_tweet_mentions.append(decoded['entities']['user_mentions'][i]['screen_name']) 
                #print "nf_tweet_mentions", nf_tweet_mentions
                nf_tweet_mentions = [element.lower() for element in nf_tweet_mentions]
                nf_words.append(nf_tweet_mentions) 

            num_list = [0]
            if nf_pos_first_hashtag > 0:
                num_list.append(nf_pos_first_hashtag)
            if nf_pos_first_url > 0:
                num_list.append(nf_pos_first_url)
            if nf_pos_first_mention > 0:
                num_list.append(nf_pos_first_mention)
            nf_pos_first_entity = min(num_list)
            #print "nf_pos_first_entity", nf_pos_first_entity

            nf_user = decoded['user']['screen_name']
            #print "nf_user", nf_user

            nf_timestamp = decoded['timestamp_ms']
            #print "nf_timestamp", nf_timestamp
            
            #file.write(nf_tweet_filtered_text, nf_positiu, nf_count_hashtags, nf_pos_first_hashtag)


    except (KeyError):
        print "JSON format error", KeyError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (ValueError):
        print "JSON format error", ValueError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (TypeError):
        print "JSON format error", TypeError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
        

print total, count, index, exception
print max(counts), min(counts), sum(counts)/len(counts)
print max(words), min(words), sum(words)/len(words)
#print corpus


['terrorism']
12 9 1 ** perezhilton heartbreaking that i teach kids day one love respect everyone this still terrorism towards gays
109 79 2 ** rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r
138 100 3 ** rt nero this terrorism it islamism it extremism this islam
282 189 4 ** rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r
612 432 5 ** rt andreagrimes i fear officials calling terrorism
830 586 6 ** rt thegopreport looks like jeff sessions right attacks we need trump lead us amp strong terrorism http
919 655 7 ** potus realdonaldtrump hillaryclinton radical islamic terrorism hate wow blinded ideology made us unsafe fact
7 710 999 0
156 7 95
28 1 11


In [42]:
file.close()

In [43]:
#vectorizer = CountVectorizer(min_df=1, stop_words='english')
vectorizer = CountVectorizer(stop_words='english')
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [45]:
X = vectorizer.fit_transform(corpus)
X

<710x3374 sparse matrix of type '<type 'numpy.int64'>'
	with 6145 stored elements in Compressed Sparse Row format>

In [46]:
print (vectorizer.vocabulary_)
vocab = vectorizer.get_feature_names()
print vocab
print len(vocab)

{u'happeningnow': 1238, u'bear': 257, u'ladyteenintl': 1617, u'yellow': 3312, u'bob': 333, u'xdsomrw': 3282, u'sleep': 2680, u'asian': 171, u'hate': 1256, u'swag': 2838, u'buddy': 382, u'tweet': 3046, u'ojoguk': 2066, u'qoaqui': 2343, u'voted': 3151, u'wkbgjt': 3237, u'lord': 1726, u'sorry': 2720, u'pride': 2299, u'worth': 3260, u'fists': 1035, u'kvng': 1604, u'rescue': 2440, u'compassion': 572, u'pilotepisode': 2212, u'delo': 715, u'pigment': 2208, u'updates': 3090, u'lecture': 1658, u'creditors': 622, u'trailer': 3002, u'tickets': 2952, u'school': 2568, u'bestfriend': 279, u'classiccrist': 520, u'fuq': 1095, u'companies': 571, u'dotcom': 793, u'beuqzsl': 282, u'voteblue': 3150, u'nigerians': 1995, u'countryday': 605, u'oscuddly': 2112, u'shockproof': 2638, u'wzlh': 3276, u'ouvxhvo': 2120, u'yiayia': 3317, u'snstats': 2697, u'everygiri': 935, u'miller': 1868, u'bacon': 218, u'ejemplo': 859, u'pulse': 2326, u'rusty': 2517, u'street': 2787, u'akbar': 66, u'znev': 3362, u'blue': 324, u'p

In [47]:
print X.toarray()

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [48]:
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print tfidf.fit_transform(vectorizer.fit_transform(corpus)).toarray()

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [50]:
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(corpus)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
# the word topic should be the most present
for tag, count in zip(vocab, dist):
    print count, tag

1 aaasuccessmeet
2 ab
1 abbyfoster
1 ability
4 able
1 aborito
1 abortion
1 aboutbillions
1 abroad
2 absolute
6 absolutely
2 abt
1 abusing
2 ac
1 acc
1 accent
1 accept
1 accepting
1 access
1 accomplish
1 aceptar
8 act
2 action
1 actions
3 actively
1 activism
1 acto
1 actor
4 actually
1 aczv
1 adam
3 add
1 added
1 addicted
2 addresses
1 adeola
1 admit
1 adores
1 adult
1 advances
1 advocate
2 af
4 affected
3 affos
1 afghan
1 afn
1 afraid
1 african
1 ag
1 agape
1 age
1 agenda
1 aggravating
1 aggressively
2 ago
2 agree
2 agreed
1 agruett
1 ahpar
1 ahrbpg
1 ai
1 aint
1 airplane
1 aishatyler
1 ajallen
1 aka
1 akbar
1 akudoamazing
1 akus
1 al
1 alanajongco
1 alaw
1 album
1 alcohol
1 alduballmylife
1 aldubhashtagsq
1 alecrocelramos
3 alert
1 alerts
2 alessiacara
1 ali
1 alien
1 alla
1 allegedly
1 allegiance
1 allow
2 allowed
1 allowing
1 almighty
2 alright
1 als
1 alvaradoailin
1 alvfwnyvca
1 alyraep
1 alyssa
1 amandaseales
1 amasyn
3 amazing
1 amb
1 ambassadors
1 amdres
12 america
4 american
2

In [52]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
arraylabel = np.asarray(label)
print arraylabel, 
print len(arraylabel)
print len(corpus)


Training the random forest...
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [47]:
forest = forest.fit( train_data_features, np.asarray(label))

ValueError: Number of labels=710 does not match number of samples=1430